# EOEPCA Resource Discovery Validation and Usage Notebook

## Setup

In [ ]:
import os
import requests
import json
from pathlib import Path

from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.opensearch import OpenSearch
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link
import folium

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [ ]:
load_eoepca_state()

In [ ]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'

print(f"Resource Discovery URL: {resource_discovery_domain}")

## Validate Resource Discovery Endpoints

In [ ]:
endpoints = [
    ("Landing Page", resource_discovery_domain),
    ("Swagger UI", f"{resource_discovery_domain}/openapi?f=html"),
    ("Collections", f"{resource_discovery_domain}/collections"),
    ("Conformance", f"{resource_discovery_domain}/conformance"),
    ("CSW GetCapabilities", f"{resource_discovery_domain}/csw?service=CSW&version=2.0.2&request=GetCapabilities"),
    ("STAC", f"{resource_discovery_domain}/stac")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

## Ingesting a Sample Record using STAC

In [ ]:
sample_record_path = Path("example-item.json") # You will see this very simple sample file locally, feel free to experiment with it.
sample_collection = 'metadata:main'

ingest_url = f"{resource_discovery_domain}/collections/{sample_collection}/items"

with open(sample_record_path, 'r') as file:
    record = file.read()
record_json = json.loads(record)
record_id = record_json['id']
headers = {
    "Content-Type": "application/geo+json",
    "Accept": "application/json",
}
response = requests.post(ingest_url, headers=headers, data=record)

print(f"POST {ingest_url} - {response.status_code}")

### Verify Record Ingestion

In [ ]:
metadata_collection_url = f"{resource_discovery_domain}/collections/metadata:main/items"
response = requests.get(metadata_collection_url)

items = response.json().get("features", [])

for item in items:
    if item.get("id", "") == record_id:
        print("✅ Sample record successfully ingested and discoverable!")
        break

print(f'View the item in the catalogue: {resource_discovery_domain}/collections/{sample_collection}/items/{record_id}')

# CSW

In [ ]:
system_catalogue_endpoint = f'{resource_discovery_domain}/csw'
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

In [ ]:
print(f"CSW GetCapabilities: {csw.identification.title}")
print(f"CSW Version: {csw.identification.version}")
print(f"CSW Service Type: {csw.identification.type}")
print(f"CSW Operations: {[op.name for op in csw.operations]}")


In [ ]:
csw.getrecords2(maxrecords=10)
csw.results

for rec in csw.records:
   print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

In [ ]:
bbox_query = BBox(
    [
        -180.0,
        -90.0,
        180.0,
        90.0
    ]
)

begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2012-09-10 00:00')

filter_list = [
    And(
        [
            bbox_query,
            begin
        ]
    )
]

csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results


In [ ]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'metadata:main')
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

In [ ]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

# OpenSearch

In [ ]:
opensearch_endpoint = f'{resource_discovery_domain}/csw?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'
os = OpenSearch(opensearch_endpoint)

In [ ]:
print(f"Short name: {os.description.shortname}")
print(f"Long name: {os.description.longname}")
print(f"Description: {os.description.description}")
print(f"URLs: {os.description.urls}")

In [ ]:
results = os.search('application/atom+xml')
len(results)

# OGC API Records

In [ ]:
w = Records(resource_discovery_domain)

In [ ]:
print(f"URLs: {w.url}")
print(f"Conformance: {w.conformance()}")
print(f"API Version: {w.api()}")

In [ ]:
records = w.records()
print(f"Records: {records}")

In [ ]:
my_catalogue = w.collection('metadata:main')
print(f"Catalogue: {my_catalogue['id']}")
print(f"Queryable: {w.collection_queryables('metadata:main')}")

In [ ]:
my_catalogue_query = w.collection_items('metadata:main')
print(f"Matched records: {my_catalogue_query['numberMatched']}")
print(f"Metadata of first result: {my_catalogue_query['features'][0]['properties']}")


In [ ]:
spatial_query = w.collection_items('metadata:main', bbox=[-180.0, -90.0, 180.0, 90.0])
print(f"Matched records: {spatial_query['numberMatched']}")
print([f['id'] for f in spatial_query['features']])

In [ ]:
my_catalogue_datetime_query = w.collection_items(
    'metadata:main',
    cql={
        'op': '>',
        'args': [
            {'property': 'datetime'},
            '2026-01-02T00:00:00Z'
        ]
    }
)

print("Matched items:", my_catalogue_datetime_query['numberMatched'])
print("Item IDs:", [f['id'] for f in my_catalogue_datetime_query['features']])

## STAC Items Search

In [ ]:
search_url = f"{resource_discovery_domain}/stac/search"
search_payload = {
    "bbox": [-180.0, -90.0, 180.0, 90.0],
    "limit": 3
}

response = requests.post(search_url, json=search_payload)
print(f"STAC search status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))